# Train and Fine-Tune Sentence Transformers Models

## Defining the model object


In [1]:
#importing necessary libraries

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch.utils.data import DataLoader
from datasets import load_dataset
import random


In [2]:
'''creating a model object with a simple architecture of a pretrained model and a pooling layer after it'''

## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

## Preparing the dataset for training a Sentence Transformers model


Looks like dataset['train']['set'] is used as the training data. It is a list of dictionaries. In the following section the dataloader will be defined using the musiccaps data.

In [4]:
# Loading the MusicCaps dataset from HuggingFace
msd_dataset = load_dataset('seungheondoh/LP-MusicCaps-MSD')

In [5]:
train = pd.DataFrame(msd_dataset['train'])
test = pd.DataFrame(msd_dataset['test'])
valid = pd.DataFrame(msd_dataset['valid'])

In [6]:
data = pd.concat([train, test, valid], ignore_index= True)
# data

In [7]:
caption_columns = ['tag', 'caption_writing', 'caption_paraphrase']
df = data[caption_columns]
train = train[caption_columns]

In [8]:
index_random = list(range(0, (len(df))))
random.shuffle(index_random)

print(index_random[:5])
print(len(index_random))

[39669, 389460, 482, 269353, 327135]
513977


In [9]:
# neg_column = []

# for i in range(len(train)):
    # neg_column.append(df['caption_writing'][index_random[i]])

In [10]:
# train.caption_writing.value_counts()

In [11]:
# Counter(neg_column)

In [12]:
i = 0
j = len(df)-1
neg = []

for i in range(len(train)):
    pos = train['caption_writing'][i]    
    random_index = index_random[i]
    neg_sentence = df['caption_writing'][random_index]       

    while pos == neg_sentence:
        neg_sentence = df['caption_writing'][j]
        j = j - 1
    
    neg.append(neg_sentence)
    
train['neg'] = neg

In [13]:
train.neg.value_counts()

neg
This catchy pop rock tune will have you tapping your feet and singing along in no time.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   48
This catchy and upbeat pop rock song will have you tapping your feet and singing along in no time.                                                                                                                                                                                                                                                                                                     

In [30]:
music_corpus.to_csv('/home/mendu/Thesis/data/musiccaps/triplets_train_lp_musiccaps_msd.csv')

In [15]:
train['same'] = train['caption_writing'] == train['neg']

In [16]:
train.same.value_counts()

same
False    444865
Name: count, dtype: int64

In [19]:
train

,tag,caption_writing,caption_paraphrase,neg,same
0,"[cathartic, earnest, punk, urgent, confident, ...","This aggressive, confrontational, and energeti...",This alternative indie rock anthem brings a ro...,This aggressive and negative pop rock song has...,False
1,"[hanging out, giddy, alternative pop rock, roc...",This alternative indie rock song combines gidd...,Get ready to experience a roller-coaster of em...,This new single is an R&B slow burner that wil...,False
2,"[laid back mellow, reflective, dramatic, organ...",This song's Nashville Sound Countrypolitan ble...,This Nashville sound countrypolitan ballad is ...,Get ready to hit the open road and feel the wi...,False
3,"[alternative indie rock, pop rock, indie]",Get lost in the captivating sound of alternati...,This song is a captivating blend of alternativ...,Get ready to hit the open road with the energe...,False
4,"[reflective, delicate, soft rock, day driving,...",Take a nostalgic journey down memory lane with...,This song is a sentimental journey through a S...,Experience an intimate and romantic journey wi...,False
...,...,...,...,...,...
444860,[pop rock],Get ready to experience the perfect fusion of ...,This dynamic pop rock anthem is the perfect fu...,This adult contemporary R&B masterpiece is an ...,False
444861,"[psychedelic pop, folk rock, pop rock, psyched...",This mind-bending tune combines the raw energy...,Embodying the vintage sounds of psychedelic ga...,This smooth and vibrant tune showcases the inc...,False
444862,"[big band, laid back mellow, easy listening, j...",This easy listening instrumental pop track is ...,This instrumental pop track boasts a laid back...,This alternative track has a moody melody and ...,False
444863,"[club dance, electronica, techno, electronic, ...",This electronic techno track incorporates elem...,Get lost in the hypnotic rhythms of this elect...,This hauntingly beautiful piece transports the...,False


In [128]:
index = 99

print('query:',train['caption_writing'][index])
print('pos:', train['caption_paraphrase'][index])
print('neg:', train['neg'][index])

query: Blending elements of blues rock and classic rock with a pop rock twist, this poignant and bittersweet ballad delivers a message of earnestness and gentle reflection, accented by wry lyrics, powerful guitar riffs, and an amiable good-natured vibe that is both laid back and mellow, all while maintaining an organic feel and hint of southern rock influence.
pos: This heartfelt tune is a blend of blues rock and classic rock with a touch of pop rock, featuring an amiable good-natured vibe and a laid-back mellow sound that is both earnest and gentle. The wry lyrics and soulful guitar give it an organic feel, combining elements of blues and rock into a bittersweet and poignant southern rock anthem.
neg: This sophisticated pop rock song from a contemporary singer songwriter is both intense and amiable good natured, evoking a wistful and reflective mood in its plaintive guitar instrumental. With a blend of contemporary folk and adult contemporary influences, it's perfect for a lazy Sunday

In [27]:
train['caption_paraphrase'][3]

'This Nashville sound countrypolitan ballad is a stripped back reflection on the weary, poignant moments we all experience in our searching, yearning lives, blending rollicking and amiable good natured drinking anthems with wistful, melancholy country folk ballads. With irreverent humor and an earthy, traditional country feel, this mellow yet dramatic track is perfect for late night, guys night out playlists, with reflective, plaintive lyrics that will leave you feeling both bittersweet and longing for more.'

In [29]:
train['neg'][3]

'Get ready to hit the open road with the energetic and exuberant sound of sweet and passionate tejano and latin pop, perfect for hanging out and having fun with friends while embracing Mexican traditions in a playful, summery, and carefree way that will fill your heart with joy and cheer.'

In [31]:
# i = 0
# neg = []

# for i in range(len(df)):
#     pos = df['caption_writing'][i]
#     tags = df['tag'][i]
#     # min = len(pos) - 30
#     # max = len(pos) + 31
#     while True:     
#         random_row = random.randint(0, (len(df)-1))
#         random_tag = df['tag'][random_row]
#         random_caption = df['caption_writing'][random_row]

#         # if (pos != random_caption) & (len(random_caption) in range(min, max)): 
#         if (pos != random_caption):
#             # add the caption to the neg list
#             neg.append(random_caption)
#             break

In [9]:
len(neg)

444865

In [17]:
df

,tag,caption_writing,caption_paraphrase,neg
0,"[cathartic, earnest, punk, urgent, confident, ...","This aggressive, confrontational, and energeti...",This alternative indie rock anthem brings a ro...,This elegant world fusion track blends laid ba...
1,"[hanging out, giddy, alternative pop rock, roc...",This alternative indie rock song combines gidd...,Get ready to experience a roller-coaster of em...,This upbeat track is the quintessential pop an...
2,"[laid back mellow, reflective, dramatic, organ...",This song's Nashville Sound Countrypolitan ble...,This Nashville sound countrypolitan ballad is ...,This upbeat pop rock tune features a catchy mi...
3,"[alternative indie rock, pop rock, indie]",Get lost in the captivating sound of alternati...,This song is a captivating blend of alternativ...,An infectious blend of Canadian pop with a Fre...
4,"[reflective, delicate, soft rock, day driving,...",Take a nostalgic journey down memory lane with...,This song is a sentimental journey through a S...,An adrenaline-pumping pop rock anthem infused ...
...,...,...,...,...
513972,"[spanish, pop rock]",Get ready to dance and sing along to the upbea...,This song is a fusion of upbeat pop rock and f...,Get lost in the mesmerizing and atmospheric so...
513973,"[trance, religious, gospel, electronic, dance]",This electronic gospel track is a euphoric dan...,This high-energy track blends electronic eleme...,This explosive song combines gritty alternativ...
513974,"[hanging out, night driving, indie rock, urgen...",This alternative indie rock anthem is fueled b...,Cruise into the night with no regard for conse...,This song blends the styles of alternative ind...
513975,[80s],This upbeat track throws it back to the 80s wi...,Introducing a nostalgia-inducing hit that soun...,This is a fiery and intense Latin rap track wi...


In [28]:
df.caption_writing[18]

'This sophisticated and refined pop rock song features a passionate and intense female vocal, perfect for a laid-back Sunday afternoon gathering or a romantic evening. With elements of vocal jazz and neo soul, the earthy and sensual tones create a warm and intimate atmosphere while still maintaining a street smart and stylish edge. The soothing and good-natured tone of the track brings a sense of calm peacefulness perfect for relaxation or introspective reflection, complemented by the gentle and wistful pop melodies. This is an elegant and organic track with a bittersweet tone that perfectly captures the complexities of life.'

In [29]:
df.caption_paraphrase[18]

'This sophisticated and refined pop rock ballad featuring a sultry female vocalist creates a calm and peaceful atmosphere perfect for a small gathering or a relaxing Sunday afternoon. The intense and passionate vocals, combined with the amiable and good-natured vibe, evoke both introspection and reflection, making it a gentle and introspective vocal jazz piece with a touch of earthy soul and neo-soul influences. The warm and elegant street-smart lyrics are perfect for a romantic evening or a laid-back mellow hangout, as the soothing and soft adult contemporary sound washes over you, leaving you feeling both sexy and bittersweet. Overall, an intimate and organic song for those moments of wistful relaxation.'

In [30]:
df.neg[18]

'This hauntingly beautiful alternative track has a gothic and atmospheric tone that creates a dark and mesmerizing sound perfect for any goth enthusiast.'

In [13]:
train

,tag,caption_writing,caption_paraphrase,neg
0,"[cathartic, earnest, punk, urgent, confident, ...","This aggressive, confrontational, and energeti...",This alternative indie rock anthem brings a ro...,An upbeat and energetic track with thumping el...
1,"[hanging out, giddy, alternative pop rock, roc...",This alternative indie rock song combines gidd...,Get ready to experience a roller-coaster of em...,This song features fast-paced rap verses that ...
2,"[laid back mellow, reflective, dramatic, organ...",This song's Nashville Sound Countrypolitan ble...,This Nashville sound countrypolitan ballad is ...,This upbeat pop song tinged with nostalgia wil...
3,"[alternative indie rock, pop rock, indie]",Get lost in the captivating sound of alternati...,This song is a captivating blend of alternativ...,This energetic song combines the best of aggre...
4,"[reflective, delicate, soft rock, day driving,...",Take a nostalgic journey down memory lane with...,This song is a sentimental journey through a S...,This high-energy punk rock track blends classi...
...,...,...,...,...
444860,[pop rock],Get ready to experience the perfect fusion of ...,This dynamic pop rock anthem is the perfect fu...,Get ready to move to the beat with powerful fe...
444861,"[psychedelic pop, folk rock, pop rock, psyched...",This mind-bending tune combines the raw energy...,Embodying the vintage sounds of psychedelic ga...,This classic rock and pop rock hybrid is a pas...
444862,"[big band, laid back mellow, easy listening, j...",This easy listening instrumental pop track is ...,This instrumental pop track boasts a laid back...,This funky soul track blends groovy basslines ...
444863,"[club dance, electronica, techno, electronic, ...",This electronic techno track incorporates elem...,Get lost in the hypnotic rhythms of this elect...,This upbeat pop rock track features captivatin...


In [14]:
music_corpus = train

In [3]:
# triplets_train_lp_musiccaps_msd.csv has 3 columns query, pos and negative and has captions of music
# music_corpus = pd.read_csv('/home/mendu/Thesis/data/musiccaps/triplets_train_lp_musiccaps_msd_old.csv', index_col = [0])

In [19]:
music_corpus.columns

Index(['tag', 'query', 'pos', 'neg'], dtype='object')

In [18]:
music_corpus.columns = ['tag', 'query', 'pos', 'neg']

In [24]:
len(str(music_corpus.neg.value_counts().head(1)))

123

In [28]:
music_corpus.columns = ['tags', 'query', 'pos', 'neg']

In [20]:
for index, row in music_corpus.iterrows():
    if index >= 5:
        break
    print(row['query'])

This aggressive, confrontational, and energetic alternative indie rock song boasts self-conscious, rowdy bravado with heavy punk and pop rock influence, filled with passionate, confident, and gutsy vocals, as well as swaggering urgency, and anguished distraught feelings. Its cathartic and rebellious lyrics, dramatic delivery, and street-smart attitude make it a perfect fit for anyone seeking a cutting-edge, alternative pop rock, punk rock, or hardcore punk sound, while also featuring a summery, knotty, volatile, and fiery new wave vibe.
This alternative indie rock song combines giddy rhythms and crunchy guitar riffs with angst-ridden lyrics, evoking anguished distraught emotions while still maintaining a playful and fun vibe that's perfect for hanging out or driving around town. The mix of punk pop and pop rock influences creates a unique sound that's somewhere between alternative pop rock and rock, making it perfect for fans of all genres.
This song's Nashville Sound Countrypolitan bl

In [3]:
music_corpus = pd.read_csv('/home/mendu/Thesis/data/musiccaps/triplets_train_lp_musiccaps_msd.csv', index_col=[0])

In [4]:
music_corpus.head()

,tags,query,pos,neg
0,"['cathartic', 'earnest', 'punk', 'urgent', 'co...","This aggressive, confrontational, and energeti...",This alternative indie rock anthem brings a ro...,This aggressive and negative pop rock song has...
1,"['hanging out', 'giddy', 'alternative pop rock...",This alternative indie rock song combines gidd...,Get ready to experience a roller-coaster of em...,This new single is an R&B slow burner that wil...
2,"['laid back mellow', 'reflective', 'dramatic',...",This song's Nashville Sound Countrypolitan ble...,This Nashville sound countrypolitan ballad is ...,Get ready to hit the open road and feel the wi...
3,"['alternative indie rock', 'pop rock', 'indie']",Get lost in the captivating sound of alternati...,This song is a captivating blend of alternativ...,Get ready to hit the open road with the energe...
4,"['reflective', 'delicate', 'soft rock', 'day d...",Take a nostalgic journey down memory lane with...,This song is a sentimental journey through a S...,Experience an intimate and romantic journey wi...


In [5]:
train_examples = []
# For agility we only 1/4 of our available data
# there are 444865 dictionaries in the train_data
# Only using 50k dictionaries
# n_examples = music_corpus.shape[0] // 8

for index, row in music_corpus.iterrows():
    # if index >= n_examples:
    #     break
    train_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))

In [6]:
len(train_examples)

444865

We wrap our training dataset into a Pytorch `Dataloader` to shuffle examples and get batch sizes.

In [7]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)

## Loss functions for training a Sentence Transformers model


In [8]:
train_loss = losses.TripletLoss(model=model)

## How to train a Sentence Transformer model


In [9]:
num_epochs = 10

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [10]:
history = model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps)
#Maybe check for model.run/ model.forward to make this a sentence embedding

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/13903 [00:00<?, ?it/s]

In [11]:
model.save(path = '/home/mendu/Thesis/data/musiccaps/new_embedding_model2',
           model_name = 'sentence_embedding_finetunned_on_musiccaps',
           train_datasets = ['triplets_train_lp_musiccaps_msd'])

In [12]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [13]:
# Load the fine-tuned model
# model_ = SentenceTransformer('/home/mendu/Thesis/data/musiccaps/embedding_model')

In [14]:
# model_